In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
from tqdm import tqdm

In [8]:
data_path='./data/'
training_filename = data_path + 'training_data.txt'
print('Loading training dataset...')
train_data = pd.read_table(training_filename, delimiter='*')
train_data.drop(columns=['image_names'],inplace=True)
# train_data = train_data.values.tolist()
print(len(train_data))
x_train=train_data['tweets'].values
y_train=train_data['hashtags'].apply(lambda x:x.split()).values
print(x_train[0])
print(y_train[0])

Loading training dataset...
51172


/home/eric/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead.
  after removing the cwd from sys.path.


air feel smell fresh brew coffe
['me', 'night', 'fun', 'happy', 'coffee', 'love', 'amazing', 'cute', 'day', 'likeforfollow', 'likeforlikes', 'woman', 'instagood', 'mood', 'white', 'black']


In [9]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(y_train)

In [10]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])

In [11]:
validation_filename = data_path + 'validation_data.txt'
validation_data = pd.read_table(validation_filename, delimiter='*')
validation_data.drop(columns=['image_names'],inplace=True)
# validation_data = validation_data.values.tolist()
print(len(validation_data))
X_test=validation_data['tweets'].values
Y_test=validation_data['hashtags'].values
print(X_test[0])
print(Y_test[0])

5686
instead turkey got quail stuf preserv truffl pea shoot yummi gamey
foodie food foodporn foodblogger rich christmaseve


/home/eric/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [12]:
classifier.fit(x_train, Y)
predicted = classifier.predict(X_test)
all_labels = mlb.inverse_transform(predicted)

In [18]:
print(len(all_labels))
for item in all_labels[3]:
    print(item)
    


5686
fashion
fit
follow
gymlife


In [19]:
with open('svm_predict.txt','w') as f:
    for i in range(len(all_labels)):
        hashtags=[]
        for item in all_labels[i]:
            hashtags.append(item)
        f.write(' '.join(hashtags))
        f.write('\n')

In [4]:
import pickle
word_to_id_filename='word_to_id.p'
id_to_word_filename='id_to_word.p'
word_to_id = pickle.load(open(data_path +
                                           word_to_id_filename, 'rb'))
id_to_word = pickle.load(open(data_path +
                                           id_to_word_filename, 'rb'))

In [5]:
import numpy as np
max_len=1003
def to_category_vector(texts,max_len):
    vector = np.zeros(max_len).astype(np.float32)
    for word in texts :
        vector[word_to_id[word]]=1.0
    return vector
document_Y = []
test_y=[]
for example in tqdm(y_train):
    arr=example.strip().split()
    document_Y.append(to_category_vector(arr,max_len))
document_Y=np.array(document_Y)
for example in tqdm(Y_test):
    arr=example.strip().split()
    test_y.append(to_category_vector(arr,max_len))
test_y=np.array(test_y)

100%|██████████| 5686/5686 [00:00<00:00, 144840.23it/s]


In [7]:
print(document_Y.shape)
print(document_Y[1])

(51172, 1003)
[0. 0. 0. ... 0. 0. 0.]


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

NB_resuts=pd.DataFrame()
# mlb = MultiLabelBinarizer()
# Y = mlb.fit_transform(y_train)

# classifier = Pipeline([
#     ('vectorizer', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', OneVsRestClassifier(LinearSVC()))])

# classifier.fit(x_train, Y)
# predicted = classifier.predict(X_test)
# all_labels = mlb.inverse_transform(predicted)

# for item, labels in zip(X_test, all_labels):
#     print('{0} => {1}'.format(item, ', '.join(labels)))

# Define a pipeline combining a text feature extractor with multi lable classifier
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for i in tqdm(range(3,1004)):
#     print('... Processing {}'.format(id_to_word[i]))
    # train the model using X_dtm & y
#     print(x_train.shape)
#     print(document_Y[:,i].shape)
    NB_pipeline.fit(x_train, document_Y[:,i])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    resuts[id_to_word[i]]=prediction
#     break
NB_resuts.to_csv("result.csv")
#     print(prediction.shape)
#     print(test_y.shape)
#     print('Test accuracy is {}'.format(accuracy_score(test_y[:,i], prediction)))

100%|█████████▉| 998/1001 [38:57<00:07,  2.34s/it]/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0.0 is present in all training examples.
  str(classes[c]))


KeyError: 1001

In [ ]:
svc_resutls=pd.DataFrame()
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for i in tqdm(range(3,1004)):
#     print('... Processing {}'.format(id_to_word[i]))
    # train the model using X_dtm & y
#     print(x_train.shape)
#     print(document_Y[:,i].shape)
    NB_pipeline.fit(x_train, document_Y[:,i])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    svc_resutls[id_to_word[i]]=prediction
#     break
svc_resutls.to_csv("svc_resutls.csv")

In [ ]:
LR_resutls=pd.DataFrame()
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for i in tqdm(range(3,1004)):
#     print('... Processing {}'.format(id_to_word[i]))
    # train the model using X_dtm & y
#     print(x_train.shape)
#     print(document_Y[:,i].shape)
    NB_pipeline.fit(x_train, document_Y[:,i])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    LR_resutls[id_to_word[i]]=prediction
#     break
LR_resutls.to_csv("LR_resutls.csv")